In [1]:
import pandas as pd

#pd.read_csv()  is for csv like excel like files
#pd.read_table() is for text files and then you can directly read the data in
df = pd.DataFrame(pd.read_table('ospC-NE.diff'))
df
#converter = pd.read_csv('ospC-NE.diff')

#converter

,seq_1,seq_2,len_gapless,len_variable,diff_gapless,diff_variable,len_aln,identitty,fac_diff,frac_gapless,frac_variable
0,A,B,209,115,40,43,217,80.8612,0.1914,0.1914,0.3739
1,A,C,209,115,38,40,217,81.8182,0.1818,0.1818,0.3478
2,A,D,210,115,47,48,217,77.6190,0.2238,0.2238,0.4174
3,A,E,209,115,55,56,217,73.6842,0.2632,0.2632,0.4870
4,A,F,208,115,47,50,217,77.4038,0.2260,0.2260,0.4348
...,...,...,...,...,...,...,...,...,...,...,...
115,M,T,212,115,46,47,217,78.3019,0.2170,0.2170,0.4087
116,M,U,210,115,44,49,217,79.0476,0.2095,0.2095,0.4261
117,N,T,211,115,54,56,217,74.4076,0.2559,0.2559,0.4870
118,N,U,209,115,52,58,217,75.1196,0.2488,0.2488,0.5043


In [2]:
'''
import numpy as np 


for index, row in df.iterrows():
    #print("index :", index)
    #print("row value : ", row)
    print("row A :" , row[0])
    print("ordinate : ",  row[0].lower())
    print("new orifnate : ", ord(row[0].lower())%32)
    print("row A :" , row[1])
    print("ordinate : ",  row[1].lower())
    print("new orifnate : ", ord(row[1].lower())%32)
    #print(type(row[0]))
    #print("row B : ", row[1])
    #print("fracVar : ", row[10])

'''

'\nimport numpy as np \n\n\nfor index, row in df.iterrows():\n    #print("index :", index)\n    #print("row value : ", row)\n    print("row A :" , row[0])\n    print("ordinate : ",  row[0].lower())\n    print("new orifnate : ", ord(row[0].lower())%32)\n    print("row A :" , row[1])\n    print("ordinate : ",  row[1].lower())\n    print("new orifnate : ", ord(row[1].lower())%32)\n    #print(type(row[0]))\n    #print("row B : ", row[1])\n    #print("fracVar : ", row[10])\n\n'

In [3]:
from scipy import sparse
import numpy as np
def create_dict(df):  ####this could be so much more effient... screw it... going to go with it. 
    translator =  {}
    counter = 0
    for index, row in df.iterrows():
        if row[0] not in translator.keys():
            translator[row[0]]=counter
            counter+=1
        else:
            continue
    
    translator['U']=counter
    return translator
translator = create_dict(df)
#print(len(translator))
        
def adjacency_matrix(df, translator):
       
        dicter = {}
        row = []
        col = []
        data = []

        for index, val in df.iterrows():
            row.append(translator[val[0]])  #best I have for now. I dont like doing it this way
            col.append(translator[val[1]])
            data.append(val[10])
            dicter[translator[val[0]],translator[val[1]]]=val[10]
            ## the data is actually nxn .. hence it is a square matrix nxn.. so this
            ## makes the adjacnecy matrix bidirectional
            row.append(translator[val[1]])
            col.append(translator[val[0]])
            data.append(val[10])
            dicter[translator[val[1]],translator[val[0]]]=val[10]



        shape = (len(row), len(col))
        #A = sparse.coo_matrix((data, (row, col)), shape=shape)

        return A, row, col, data, dicter
    



adjac, row, col, data, dicter = adjacency_matrix(df, translator)

def create_numpy_matrix(translator, dicter):
    a_shape = (len(translator),len(translator))
    a= np.zeros(a_shape)
    for i,j in dicter:
          a[i,j]=dicter[i,j]
          a[i,i]=1
    
    return a
numpy_matrix_form = create_numpy_matrix(translator, dicter)
print(numpy_matrix_form)



[[1.     0.3739 0.3478 0.4174 0.487  0.4348 0.487  0.4522 0.3913 0.4348
  0.4    0.487  0.4087 0.5043 0.4696 0.3913]
 [0.3739 1.     0.3304 0.4435 0.5391 0.4522 0.5652 0.4957 0.4    0.4957
  0.4087 0.513  0.513  0.4609 0.5565 0.513 ]
 [0.3478 0.3304 1.     0.5043 0.4783 0.4261 0.5565 0.4174 0.2957 0.4
  0.3739 0.4609 0.4348 0.4609 0.4696 0.4348]
 [0.4174 0.4435 0.5043 1.     0.5217 0.4435 0.4696 0.4609 0.4    0.4609
  0.4435 0.487  0.4609 0.5217 0.4261 0.5043]
 [0.487  0.5391 0.4783 0.5217 1.     0.487  0.5391 0.4696 0.4957 0.4435
  0.4174 0.5826 0.4348 0.487  0.4609 0.4696]
 [0.4348 0.4522 0.4261 0.4435 0.487  1.     0.487  0.3565 0.4783 0.3826
  0.4261 0.4696 0.4174 0.4957 0.3739 0.4783]
 [0.487  0.5652 0.5565 0.4696 0.5391 0.487  1.     0.5217 0.5826 0.5304
  0.5304 0.5391 0.4609 0.4348 0.4609 0.5565]
 [0.4522 0.4957 0.4174 0.4609 0.4696 0.3565 0.5217 1.     0.4696 0.2609
  0.4609 0.3565 0.4783 0.5391 0.3913 0.4783]
 [0.3913 0.4    0.2957 0.4    0.4957 0.4783 0.5826 0.4696 1.     0.

In [13]:
import numpy as np


eigenvalues, eigenvectors = np.linalg.eig(numpy_matrix_form)
sorted_eigenvalues = sorted(eigenvalues)
eigenvalues_spacing = []
for i in range(len(eigenvalues)-1):
    eigenvalues_spacing.append(abs(eigenvalues[i]-eigenvalues[i+1]))

print("THIS IS FOR FRACTIONAL VARIABILITY FEATURE :::")
print(" The eigenvalues are : \n", eigenvalues)
print(" The eigenvectors are : \n", eigenvectors)
print("The sorted eigenvalues are: \n", sorted_eigenvalues)
print("The spacings between each eigenvalue are: \n", eigenvalues_spacing)






THIS IS FOR FRACTIONAL VARIABILITY FEATURE :::
 The eigenvalues are : 
 [4.62001664 0.54665329 0.58903494 0.64382797 0.88731551 0.87511851
 0.71488272 0.7301235  0.84841829 0.74162582 0.83421254 0.82452256
 0.80496747 0.79822066 0.76636671 0.77469286]
 The eigenvectors are : 
 [[-0.23497941  0.23059668 -0.13076704  0.09395629 -0.00825988  0.0377304
  -0.08792693 -0.44961491 -0.54452417 -0.02970266 -0.4689886   0.14867285
   0.13768683 -0.30490958 -0.05032501 -0.02006649]
 [-0.25352855  0.37206908 -0.20863193  0.22021435 -0.21703868 -0.23389282
   0.25678665  0.1349727   0.41522995 -0.21999324 -0.05620256 -0.21515036
  -0.02346332 -0.44879702  0.17877244  0.12291202]
 [-0.23397745  0.27877428 -0.31117075 -0.09266264  0.36407685  0.53510306
   0.1176993   0.04466171  0.29359469 -0.11587124  0.0431333   0.23027561
   0.14110582  0.11297822 -0.29361127 -0.23237252]
 [-0.25372168 -0.01952562  0.06737302  0.43807541  0.22079723  0.321597
  -0.45443089  0.08656942  0.06679288  0.10669989  0.0

In [10]:

def create_dict(df): 
    translator =  {}
    counter = 0
    for index, row in df.iterrows():
        if row[0] not in translator.keys():
            translator[row[0]]=counter
            counter+=1
        else:
            continue
    
    translator['U']=counter
    return translator
translator = create_dict(df)

        
def adjacency_matrix(df, translator):
       
        dicter = {}

        for index, val in df.iterrows():
          
            dicter[translator[val[0]],translator[val[1]]]=val[9]
            dicter[translator[val[1]],translator[val[0]]]=val[9]

        shape = (len(row), len(col))
        
        return  dicter

dicter = adjacency_matrix(df, translator)

def create_numpy_matrix(translator, dicter):
    a_shape = (len(translator),len(translator))
    a= np.zeros(a_shape)
    for i,j in dicter:
          a[i,j]=dicter[i,j]
      
          a[i,i]=1
    
    return a
numpy_matrix_form = create_numpy_matrix(translator, dicter)
print(numpy_matrix_form)



[[1.     0.1914 0.1818 0.2238 0.2632 0.226  0.25   0.2392 0.2057 0.2333
  0.2143 0.2584 0.2057 0.2548 0.2429 0.1914]
 [0.1914 1.     0.1762 0.2333 0.2788 0.2308 0.2885 0.2667 0.2143 0.2619
  0.2143 0.2762 0.2584 0.2344 0.2857 0.2536]
 [0.1818 0.1762 1.     0.2714 0.25   0.2212 0.2885 0.2286 0.1619 0.2143
  0.2    0.2524 0.2201 0.2392 0.2429 0.2153]
 [0.2238 0.2333 0.2714 1.     0.2775 0.2344 0.2356 0.2476 0.2143 0.2512
  0.2417 0.2619 0.2381 0.2679 0.2227 0.2571]
 [0.2632 0.2788 0.25   0.2775 1.     0.2596 0.2837 0.2452 0.2596 0.2344
  0.2201 0.3077 0.2249 0.25   0.2344 0.2392]
 [0.226  0.2308 0.2212 0.2344 0.2596 1.     0.2464 0.1827 0.25   0.201
  0.2249 0.2452 0.2153 0.2548 0.1866 0.244 ]
 [0.25   0.2885 0.2885 0.2356 0.2837 0.2464 1.     0.2692 0.3029 0.2692
  0.2692 0.2788 0.2429 0.2333 0.2381 0.274 ]
 [0.2392 0.2667 0.2286 0.2476 0.2452 0.1827 0.2692 1.     0.2571 0.1381
  0.2476 0.1952 0.244  0.2823 0.2    0.2392]
 [0.2057 0.2143 0.1619 0.2143 0.2596 0.25   0.3029 0.2571 1.     

In [11]:
eigenvalues, eigenvectors = np.linalg.eig(numpy_matrix_form)
sorted_eigenvalues = sorted(eigenvalues)
eigenvalues_spacing = []
for i in range(len(eigenvalues)-1):
    eigenvalues_spacing.append(abs(eigenvalues[i]-eigenvalues[i+1]))

print("THIS IS FOR FRACTIONAL GAPLESS FEATURE:::")
print(" The eigenvalues are : \n", eigenvalues)
print(" The eigenvectors are : \n", eigenvectors)
print("The sorted eigenvalues are: \n", sorted_eigenvalues)
print("The spacings between each eigenvalue are: \n", eigenvalues_spacing)

 The eigenvalues are : 
 [4.62001664 0.54665329 0.58903494 0.64382797 0.88731551 0.87511851
 0.71488272 0.7301235  0.84841829 0.74162582 0.83421254 0.82452256
 0.80496747 0.79822066 0.76636671 0.77469286]
 The eigenvectors are : 
 [[-0.23497941  0.23059668 -0.13076704  0.09395629 -0.00825988  0.0377304
  -0.08792693 -0.44961491 -0.54452417 -0.02970266 -0.4689886   0.14867285
   0.13768683 -0.30490958 -0.05032501 -0.02006649]
 [-0.25352855  0.37206908 -0.20863193  0.22021435 -0.21703868 -0.23389282
   0.25678665  0.1349727   0.41522995 -0.21999324 -0.05620256 -0.21515036
  -0.02346332 -0.44879702  0.17877244  0.12291202]
 [-0.23397745  0.27877428 -0.31117075 -0.09266264  0.36407685  0.53510306
   0.1176993   0.04466171  0.29359469 -0.11587124  0.0431333   0.23027561
   0.14110582  0.11297822 -0.29361127 -0.23237252]
 [-0.25372168 -0.01952562  0.06737302  0.43807541  0.22079723  0.321597
  -0.45443089  0.08656942  0.06679288  0.10669989  0.01137378  0.1109456
  -0.24532555  0.02643707  0

In [ ]:
'''
I fixed the negative eigenvalues by making the diagonal values equal to 1.
Now the eigenvalues are positive.
In one of the random matrix theory papers I read they mentioned how finding 
appropriates values for the diagonal is the key for stabilizing the matrix. This logic
combined with the simple maths of how eigenvalues are calculated made the 1 on
the diagonal seem plausible. 
What should be noted is that it says all sequences with A<->A , B<->B, C<->C,...,U<->U  
has a fractional variablility of 1. This could be incorrect based on what the 
fractional variablitiy feature actually means for the seuqences. 

As for why the first eigenvalue is so much larger than the rest. I still do not have
much understanding of this. But after doing some reading it says that real symmetric
matrices with random values as entries have exactly one large eigenvalue.

If you look at the code below then it can be verified that the matrix is symmetric. 
It can also be verified that the matrix does have random values. Therefore, 
making the leading eigenvector much larger than the rest.
'''

In [12]:

def transposeMatrix(inputMatrix, t, rows):
  
      for p in range(rows):
        
         for q in range(rows):
         
               t[p][q] = inputMatrix[q][p]

def checkingSymmetric(inputMatrix, rows):
   
      t = [[0 for q in range(len(inputMatrix[0]))]
         for p in range(len(inputMatrix))]
     
      transposeMatrix(inputMatrix, t, rows)
  
      for p in range(rows):
       
         for q in range(rows):
          
               if (inputMatrix[p][q] != t[p][q]):
               
                  return False

      return True

inputMatrix = [[6, 3, 5], [3, 2, 4], [5, 4, 6]]

if (checkingSymmetric(numpy_matrix_form, 16)):
   print("Input matrix is a Symmetric matrix")
else:
   print("Input matrix is NOT a Symmetric matrix")

Input matrix is a Symmetric matrix
